In [1]:
import jax.random as jrn
import jax.numpy as jnp
import jax.lax as jla
import jax
import random

from tqdm import trange
from jax import tree_util

from game.run import dummy_history

In [2]:
# dummy_history_jit = jax.jit(dummy_history, static_argnames=["player_total", "game_len"])

# key = jrn.PRNGKey(random.randint(0, 1000))
# dummy_history_jit(key, 10, 30);

In [3]:
# def block_until_ready(pytree):
#     return tree_util.tree_map(lambda x: x.block_until_ready(), pytree)


# for _ in trange(100000):
#     r = dummy_history_jit(key, 10, 30)
#     block_until_ready(r)

In [4]:
key = jrn.PRNGKey(random.randint(0, 1000))
batched_history = dummy_history(key, 5, 11, prob_vote=0.6)

history = {}

for k, v in batched_history.items():
    history[k] = v[-1]


history["board"], history["winner"], history["tracker"]

(DeviceArray([[2, 4],
              [2, 4],
              [1, 4],
              [1, 4],
              [1, 3],
              [1, 2],
              [1, 2],
              [1, 2],
              [1, 1],
              [0, 1],
              [0, 1]], dtype=int32),
 DeviceArray([[False,  True],
              [False,  True],
              [False,  True],
              [False,  True],
              [False, False],
              [False, False],
              [False, False],
              [False, False],
              [False, False],
              [False, False],
              [False, False]], dtype=bool),
 DeviceArray([1, 0, 1, 0, 0, 2, 1, 0, 0, 1, 0], dtype=int32))

In [5]:
player = 1

print("voted  ", *history["voted"][:, player].astype(int))
print("winner ", *history["winner"].astype(int))
print("roles  ", *history["roles"][0].astype(int))

voted   1 1 0 0 1 0 1 1 1 0 1
winner  [0 1] [0 1] [0 1] [0 1] [0 0] [0 0] [0 0] [0 0] [0 0] [0 0] [0 0]
roles   2 0 0 0 1


In [6]:
def vote_data(player: int, winner, voted, roles, **rest):
    """
    """
    role = roles[0, player]
    role = role != 0 # only check if F

    # 0: L win, 1: F win, 
    # also 0 for no winner
    wins = winner[0].argmax()
    invs = wins != role

    # iff there is a winner
    done = winner[0].sum() > 0

    acts = voted[:, player].astype(jnp.int32)
    rang = jnp.arange(acts.shape[0])

    data = jnp.zeros((*acts.shape, 2))
    data = data.at[rang, acts].set(1)

    data = jla.select(invs, 1 - data, data)
    data = jla.select(done, data, data.at[:].set(0.5))

    return data


vote_data(player, **history)

DeviceArray([[1., 0.],
             [1., 0.],
             [0., 1.],
             [0., 1.],
             [1., 0.],
             [0., 1.],
             [1., 0.],
             [1., 0.],
             [1., 0.],
             [0., 1.],
             [1., 0.]], dtype=float32)

In [7]:
print("presi", *history["presi"].astype(int))
print("shown", *history["presi_shown"].astype(int))
print("given", *history["chanc_shown"].astype(int))

presi 1 0 3 2 1 0 4 3 2 1 0
shown [0 0] [1 2] [0 0] [1 2] [1 2] [0 0] [0 0] [2 1] [1 2] [0 0] [0 3]
given [0 0] [1 1] [0 0] [0 2] [1 1] [0 0] [0 0] [1 1] [1 1] [0 0] [0 2]


In [15]:
def presi_discard_data(player: int, roles, winner, presi_shown, chanc_shown, **_):
    """
    """
    role = roles[0, player]
    role = role != 0 # only check if F

    # 0: L win, 1: F win, 
    # also 0 for no winner
    wins = winner[0].argmax()
    invs = wins != role

    # TODO could be [0, 0]
    acts = presi_shown - chanc_shown
    acts = acts.argmax(axis=-1)

    skip = presi_shown.sum(axis=-1) == 0

    


    return acts, skip.astype(int)


presi_discard_data(player, **history)

(DeviceArray([0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1], dtype=int32),
 DeviceArray([1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0], dtype=int32))

In [9]:
# model(**history, acts=acts, role=role) -> wins

In [10]:
# player, {hisotry x ...} -> history x action (wie gut)

